## Database Engineering
-----
- Use SQLAlchemy to model the table schemas and create a sqlite database for the tables.
- One table for the 'measurements' and one for the 'stations'.

In [1]:
# Dependencies 
import pandas as pd
import numpy as np
import os

import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Text, ForeignKey

In [2]:
# Create a Database Connection
engine = create_engine('sqlite:///hawaii.sqlite')

In [3]:
# Define the Measurement class

Base = declarative_base()

class Measurement(Base):
    __tablename__ = "measurement"
    
    index = Column(Integer, primary_key=True)
    station = Column(String(25), ForeignKey('stations.station'))
    date = Column(Text)
    prcp = Column(Float) 
    tobs = Column(Integer()) 

    def __repr__(self):
        return f"index={self.index}, station={self.station}"
    
# Define the Stations class
class Stations(Base):
    __tablename__ = "stations"
    
    station = Column(String(25), primary_key=True)
    name = Column(String(255))
    latitude = Column(Float) 
    longitude = Column(Float) 
    elevation = Column(Float)
    
    def __repr__(self):
        return f"stations={self.station}, name={self.name}"

In [4]:
# Use `create_all` to create the demographics table in the database

Base.metadata.create_all(engine)
engine.table_names()

['measurement', 'stations']

In [5]:
def add_table_data(engine, table, csvfile):
    
    # Create a connection to the engine called `conn`
    conn = engine.connect()
    
    # Read CSV files into a pandas DataFrame
    data_to_df = pd.read_csv(csvfile)
    
    # Use Orient='records' to create a list of data to write
    data = data_to_df.to_dict(orient="records")

    # Use MetaData from SQLAlchemy to reflect the tables
    metadata = MetaData(bind=engine)
    metadata.reflect()
    
 
    # Use 'table.delete()' to remove any existing data.
    conn.execute(table.delete())
    
    # Use `table.insert()` to insert the data into the table
    conn.execute(table.insert(), data)
    
    
add_table_data(engine, Measurement.__table__, "data/clean_hawaii_measurements.csv")
add_table_data(engine, Stations.__table__, "data/clean_hawaii_stations.csv")

In [6]:
# Test that the insert works by fetching the first 10 rows for the stations table. 
engine.execute("select * from stations limit 10").fetchall()

[('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 ('USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 ('USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 ('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 ('USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9),
 ('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 ('USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]

In [7]:
# Test that the insert works by fetching the first 10 rows for the measurement table. 
engine.execute("select * from measurement limit 10").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-06', 0.0, 73),
 (6, 'USC00519397', '2010-01-07', 0.06, 70),
 (7, 'USC00519397', '2010-01-08', 0.0, 64),
 (8, 'USC00519397', '2010-01-09', 0.0, 68),
 (9, 'USC00519397', '2010-01-10', 0.0, 73),
 (10, 'USC00519397', '2010-01-11', 0.01, 64)]